In [ ]:
import sys
import pandas as pd

sys.path.append('.')
data_daily = pd.read_csv('./data/48_Industry_Portfolios_Daily.csv', index_col=0)

# Trim the data
data_daily = data_daily.iloc[('197401' <= data_daily.index.values.astype(str)) & (data_daily.index.values.astype(str) <= '202012')]

# Convert the index to datetime
data_daily.index = pd.to_datetime(data_daily.index, format='%Y%m%d')
daily_date = data_daily.index.date

# The rate of return matrix, fill the missing values with 100%
Return = data_daily.values.T.astype(float)
Return[Return < -99]
Gross_return = (Return + 100) / 100 # Gross Return

print(Gross_return.shape)

# Check the first few rows of the dataset
data_daily.head()

In [ ]:
from ucb import UCB

window_size = 120
ucb = UCB(Gross_return, window_size)
#ucb.algorithm()
ucb.test()
